In [41]:
import geopandas as gpd
# load reference scenes to find out their paths
reference_scenes = gpd.read_file("/data/users/Public/jonathanbahlmann/coherence-docs/src/reference_scenes.geojson")
# load bursts we want to process
bursts_to_process = gpd.read_file("/data/users/Public/jonathanbahlmann/coherence-docs/src/processing.geojson")

# go per scene to process, dissolve to extract ids
combinations = bursts_to_process.dissolve(["id", "subswath", "ref_scene"], as_index = False)

parel_list = []
paths = bursts_to_process.loc[:,"path"].unique()
"""
for path in paths:
    scene_dict = {"path": paths, 
                  "IW1": {"ref": None, "path": None, "ref_min": None, "ref_max": None, "sce_min": None, "sce_max": None}, 
                  "IW2": {"ref": None, "path": None, "ref_min": None, "ref_max": None, "sce_min": None, "sce_max": None},
                  "IW3": {"ref": None, "path": None, "ref_min": None, "ref_max": None, "sce_min": None, "sce_max": None}}

    scene_dict["subs"]["IW1"]["ref"]
    scene_dict["subs"]["IW1"]["path"]
    scene_dict["subs"]["IW1"]["ref_min"]
    scene_dict["subs"]["IW1"]["ref_max"]
    scene_dict["subs"]["IW1"]["sce_min"]
    scene_dict["subs"]["IW1"]["sce_max"]
    """
combinations.to_dict()

# each scene and ref_scene could be added together with merge, but not across ref_scenes probably

# print(bursts_to_process.head())

# bursts_to_process.iloc[0].loc[["subswath", "burst", "id", "path", "sensor", "polarizations", "mode", "orbit_dicrection", "rel_orbit", "regular_burst_pattern", "ref_scene", "ref_min_burst", "ref_max_burst", "processing_status", "sce_min_burst", "sce_max_burst"]].to_dict()


# out into the spark parallelizer: array of dicts? what does it need: path to scene, which subswaths to process with which ref scenes, maybe like path..., subswaths

{'id': {0: '1D7F',
  1: '1D7F',
  2: '1D7F',
  3: '1D7F',
  4: '1D7F',
  5: '1D7F',
  6: '3C31',
  7: '3C31',
  8: '3C31',
  9: '3C31',
  10: '3C31',
  11: '3C31'},
 'subswath': {0: 'IW1',
  1: 'IW1',
  2: 'IW2',
  3: 'IW2',
  4: 'IW3',
  5: 'IW3',
  6: 'IW1',
  7: 'IW1',
  8: 'IW2',
  9: 'IW2',
  10: 'IW3',
  11: 'IW3'},
 'ref_scene': {0: '3785',
  1: '6471',
  2: '3785',
  3: '6471',
  4: '3785',
  5: '6471',
  6: '3785',
  7: 'ACC4',
  8: '3785',
  9: 'ACC4',
  10: '3785',
  11: 'ACC4'},
 'geometry': {0: <shapely.geometry.polygon.Polygon at 0x7f20d244ae48>,
  1: <shapely.geometry.polygon.Polygon at 0x7f20d244a2b0>,
  2: <shapely.geometry.polygon.Polygon at 0x7f20d244a2e8>,
  3: <shapely.geometry.polygon.Polygon at 0x7f20d244a390>,
  4: <shapely.geometry.polygon.Polygon at 0x7f20d244a160>,
  5: <shapely.geometry.polygon.Polygon at 0x7f20d244afd0>,
  6: <shapely.geometry.polygon.Polygon at 0x7f20d244aa20>,
  7: <shapely.geometry.polygon.Polygon at 0x7f20d244a748>,
  8: <shapely.geomet

In [24]:
dic = {"path": "/path", "subswaths": [ {"IW1": {"ref": "XCVB", "min": 4, "max": 6, "path": "/path"}}, {"IW2":  {"ref": "JKL3", "min": 1, "max": 8, "path": "/path"}}]}

In [26]:
dic["path"]
dic["subswaths"][0]["IW1"]["ref"]
dic["subswaths"][2] = {"test": "hi"}

IndexError: list assignment index out of range

In [38]:
d = {"path", "fg" }
d["path"] = "fr"
d

TypeError: 'set' object does not support item assignment

In [2]:
type({4,5,6})

set

In [4]:
type((3,4,5))

tuple

In [5]:
type([4,5,6])

list